In [1]:
import chassisml
import pickle
import cv2
import torch
import getpass
import numpy as np
import torchvision.models as models
from torchvision import transforms

## Enter credentials
Dockerhub creds and Modzy API Key

In [2]:
dockerhub_user = getpass.getpass('docker hub username')
dockerhub_pass = getpass.getpass('docker hub password')
modzy_api_key = getpass.getpass('modzy api key')

docker hub username········
docker hub password········
modzy api key········


## Prepare context dict
Initialize anything here that should persist across inference runs

In [3]:
model = models.resnet50(pretrained=True)
model.eval()

labels = pickle.load(open('./data/imagenet_labels.pkl','rb'))

transform = transforms.Compose([
            transforms.ToPILImage(),
            transforms.Resize(224),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])        

# use GPU:
device = 'cuda'
model.to(device)

# This will be passed to Chassis:
context = {
    "model": model,
    "labels": labels,
    "transform": transform,
    "device": device
}

AssertionError: Torch not compiled with CUDA enabled

## Write batch process function

* Must take list[bytes] and context dict as input
* Preprocess all inputs, run inference in batch, postprocess batch model output, return list of formatted results

In [5]:
def batch_process(inputs,context):
    
    # preprocess list of inputs
    images = []
    for input_bytes in inputs:
        decoded = cv2.imdecode(np.frombuffer(input_bytes, np.uint8), -1)
        resized = cv2.resize(decoded, (224, 224)).reshape((1,224,224,3))
        images.append(resized)
    images_arr = np.concatenate(images)
    batch_t = torch.stack(tuple(context['transform'](i) for i in images_arr), dim=0).to(context['device'])

    # run batch inference and softmax
    output = context['model'](batch_t)
    probs = torch.nn.functional.softmax(output, dim=1)
    softmax_preds = probs.detach().cpu().numpy()
    
    # postprocess
    all_formatted_results = []
    for preds in softmax_preds: 
        indices = np.argsort(preds)[::-1]
        classes = [context['labels'][idx] for idx in indices[:5]]
        scores = [float(preds[idx]) for idx in indices[:5]]
        preds = [{"class": "{}".format(label), "score": round(float(score),3)} for label, score in zip(classes, scores)]
        preds.sort(key = lambda x: x["score"],reverse=True)
        results = {"classPredictions": preds}
        all_formatted_results.append(results)
    
    # output list of formatted results
    return all_formatted_results

## Initialize Chassis Client
We'll use this to interact with the Chassis service

In [6]:
chassis_client = chassisml.ChassisClient("http://localhost:5000")

## Create and test Chassis model
* Requires `context` dict containing all variables which should be loaded once and persist across inferences
* Requires at least one of single input `process_fn` or batch input `batch_process_fn` defined above
    * If you provide `batch_process_fn`, you must also provide a `batch_size`

In [7]:
# create Chassis model
chassis_model = chassis_client.create_model(context=context,batch_process_fn=batch_process,batch_size=4)

# test Chassis model (can pass filepath, bufferedreader, bytes, or text here):
sample_filepath = './modzy/airplane.jpg'
results = chassis_model.test(sample_filepath)
print(results)

b'{"classPredictions":[{"class":"airliner","score":0.606},{"class":"crane","score":0.11},{"class":"wing","score":0.103},{"class":"chain saw, chainsaw","score":0.07},{"class":"aircraft carrier, carrier, flattop, attack aircraft carrier","score":0.048}]}'


In [8]:
# test batch locally
results = chassis_model.test_batch(sample_filepath)
print(results)

[b'{"classPredictions":[{"class":"airliner","score":0.606},{"class":"crane","score":0.11},{"class":"wing","score":0.103},{"class":"chain saw, chainsaw","score":0.07},{"class":"aircraft carrier, carrier, flattop, attack aircraft carrier","score":0.048}]}', b'{"classPredictions":[{"class":"airliner","score":0.606},{"class":"crane","score":0.11},{"class":"wing","score":0.103},{"class":"chain saw, chainsaw","score":0.07},{"class":"aircraft carrier, carrier, flattop, attack aircraft carrier","score":0.048}]}', b'{"classPredictions":[{"class":"airliner","score":0.606},{"class":"crane","score":0.11},{"class":"wing","score":0.103},{"class":"chain saw, chainsaw","score":0.07},{"class":"aircraft carrier, carrier, flattop, attack aircraft carrier","score":0.048}]}', b'{"classPredictions":[{"class":"airliner","score":0.606},{"class":"crane","score":0.11},{"class":"wing","score":0.103},{"class":"chain saw, chainsaw","score":0.07},{"class":"aircraft carrier, carrier, flattop, attack aircraft carrier

## Publish model to Modzy
Need to provide model name, model version, Dockerhub credentials, required Modzy info, AND specify gpu=True

In [25]:
response = chassis_model.publish(model_name="Torch Imagenet GPU",model_version="0.0.1",
                     registry_user=dockerhub_user,registry_pass=dockerhub_pass,
                     modzy_sample_input_path=sample_filepath,
                     modzy_api_key=modzy_api_key,gpu=True)

job_id = response.get('job_id')
final_status = chassis_client.block_until_complete(job_id)

Starting build job... Ok!


## Run sample job
Submit inference job to our newly-deploy model running on Modzy

In [ ]:
from modzy import ApiClient

client = ApiClient(base_url='https://integration.modzy.engineering/api', api_key=modzy_api_key)

input_name = final_status['result']['inputs'][0]['name']
model_id = final_status['result'].get("model").get("modelId")
model_version = final_status['result'].get("version")

# submit 16 inputs
sources = {"input_{}".format(i): {input_name: sample_filepath} for i in range(16)}
inference_job = client.jobs.submit_file(model_id, model_version, sources)
inference_job_result = client.results.block_until_complete(inference_job, timeout=None)
print(inference_job_result["results"])